In [505]:
import pandas as pd

In [586]:
x_train = pd.read_csv('dengue_features_train.csv')
x_test = pd.read_csv('dengue_features_test.csv')
y = pd.read_csv('dengue_labels_train.csv')

In [587]:
submission = pd.read_csv('submission_format.csv')

In [588]:
#x_train.sample(2)

In [589]:
#x_test.sample(2)

In [590]:
#y.sample(2)

In [591]:
#x_train[x_train['city'] == 'sj'].describe()

In [592]:
#x_train[x_train['city'] == 'iq'].describe()

# Missing values and data type conversion

In [593]:
x_train['city'].nunique()

2

In [594]:
x_train['city'].unique()

array(['sj', 'iq'], dtype=object)

In [595]:
x_train['city'].value_counts()

sj    936
iq    520
Name: city, dtype: int64

In [596]:
#sj    936, iq    520

In [597]:
x_train.fillna(method='ffill', inplace=True)

In [598]:
x_test.fillna(method='ffill', inplace=True)

In [599]:
x_test.city.replace({'sj': 0, 'iq' : 1}, inplace = True);
y.city.replace({'sj': 0, 'iq' : 1}, inplace = True);

In [600]:
x_train.city.replace({'sj': 0, 'iq' : 1}, inplace = True);

In [601]:
x_train['week_start_date'].nunique()

1049

In [602]:
#x_train.info()

# Visualisation

In [603]:
import plotly.express as px

In [604]:
df = x_train.copy()
df['total_cases'] = y.total_cases

In [605]:
df_0 = x_train[x_train['city'] == 0].copy()
df_0['total_cases'] = y.total_cases

df_1 = x_train[x_train['city'] == 1].copy()
df_1['total_cases'] = y.total_cases

In [606]:
#fig1 = px.scatter(df_0 , x="week_start_date", y="total_cases", title='Total cases in San Juan')

#fig2 = px.scatter(df_1 , x="week_start_date", y="total_cases", title='Total cases in Iquitos')

#fig1.show()
#fig2.show()

In [607]:
fig = px.scatter(df , x="week_start_date", y="total_cases", color='city', title='Total cases per city')

fig.show()

# Modelling with XGBoost

In [608]:
import xgboost as xgb

## Simple XGB model

In [609]:
xg_reg = xgb.XGBRegressor(objective='reg:squarederror')

In [610]:
del x_train['week_start_date']
del x_test['week_start_date']

In [611]:
xg_reg.fit(x_train,y.total_cases)

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [532]:
from sklearn.metrics import mean_absolute_error

In [533]:
mean_absolute_error(y.total_cases, xg_reg.predict(x_train))

8.475054539883367

In [534]:
mean_absolute_error(y.total_cases, xg_reg.predict(x_train).round())

8.469780219780219

In [535]:
preds = xg_reg.predict(x_test)

In [536]:
#print(preds)

In [537]:
#submission.head()

In [538]:
submission.total_cases = preds.round()

In [539]:
submission.total_cases = submission.total_cases.astype(int)

In [540]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 4 columns):
city           416 non-null object
year           416 non-null int64
weekofyear     416 non-null int64
total_cases    416 non-null int64
dtypes: int64(3), object(1)
memory usage: 13.1+ KB


In [541]:
submission.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,6
1,sj,2008,19,3
2,sj,2008,20,6
3,sj,2008,21,10
4,sj,2008,22,10


In [542]:
submission.to_csv('mysubmissionXGB.csv', index = False)

## XGB with time series cross validator

In [543]:
from sklearn.model_selection import TimeSeriesSplit

In [629]:
time_split = TimeSeriesSplit(n_splits=5)

In [545]:
[(el[0].shape, el[1].shape) for el in time_split.split(x_train)]

[((246,), (242,)),
 ((488,), (242,)),
 ((730,), (242,)),
 ((972,), (242,)),
 ((1214,), (242,))]

In [546]:
from sklearn.model_selection import cross_val_score

In [630]:
%%time
cv_scores = cross_val_score(xg_reg, x_train, y.total_cases, cv=time_split, 
                            scoring='neg_mean_absolute_error', n_jobs=1) 

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version



CPU times: user 680 ms, sys: 6.65 ms, total: 687 ms
Wall time: 692 ms


In [631]:
cv_scores, cv_scores.mean()

(array([-65.33948222, -36.34110209, -21.34153189, -15.51575494,
        -10.42247363]), -29.79206895389833)

## XBG with GridSearch

In [549]:
"""params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'n_estimators': [500]
        }"""

params = {
        'min_child_weight': [1, 5],
        'gamma': [0.5, 1.5, 5],
        'subsample': [0.6, 1.0],
        'colsample_bytree': [0.6, 1.0],
        'max_depth': [3, 4, 5],
        'n_estimators': [500]
        }

In [550]:
from sklearn.model_selection import GridSearchCV

In [551]:
reg_cv = GridSearchCV(xg_reg, params, verbose=1)
reg_cv.fit(x_train,y.total_cases)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


/Users/morgan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning:

The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: F

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

[Parallel(n_jobs=1)]: Done 216 out of 216 | elapsed:  3.3min fin

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:squarederror',
                                    rand...=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'colsample_bytree': [0.6, 1.0], 'gamma': [0.5, 1.5, 5],
                         'max_depth

In [552]:
reg_cv.best_params_

{'colsample_bytree': 0.6,
 'gamma': 5,
 'max_depth': 4,
 'min_child_weight': 5,
 'n_estimators': 500,
 'subsample': 0.6}

In [553]:
xg_reg = xgb.XGBRegressor(**reg_cv.best_params_)
xg_reg.fit(x_train,y.total_cases)

[19:39:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=5, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6, verbosity=1)

In [554]:
mean_absolute_error(y.total_cases, xg_reg.predict(x_train))

2.1893391725863074

In [555]:
mean_absolute_error(y.total_cases, xg_reg.predict(x_train).round())

2.1710164835164836

In [556]:
submission.total_cases = reg_cv.predict(x_test).round()
submission.total_cases = submission.total_cases.astype(int)

In [557]:
submission.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,2
1,sj,2008,19,4
2,sj,2008,20,1
3,sj,2008,21,16
4,sj,2008,22,19


In [558]:
submission.to_csv('mysubmissionXGBwithGS.csv', index = False)

## Two XGB models (one for each city)

In [559]:
x_cityzero = x_train[x_train['city'] == 0]
x_cityone = x_train[x_train['city'] == 1]

In [560]:
y_cityzero = y[y['city'] == 0]
y_cityone = y[y['city'] == 1]

In [561]:
xg_reg.fit(x_cityzero,y_cityzero.total_cases)

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[20:28:23] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=5, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6, verbosity=1)

In [562]:
mean_absolute_error(y_cityzero.total_cases, xg_reg.predict(x_cityzero))

1.7105993882267394

In [563]:
xg_reg.fit(x_cityone,y_cityone.total_cases)

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/Users/morgan/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[20:28:29] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=5,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=5, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6, verbosity=1)

In [564]:
mean_absolute_error(y_cityone.total_cases, xg_reg.predict(x_cityone))

0.5860276811971115

In [566]:
#x_test[x_test['city']==0]

## Compare Train and Test data set

In [584]:
train = x_train.copy()

In [568]:
train['train_or_test'] = 1

In [585]:
test = x_test.copy()

In [570]:
test

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,0,2008,18,-0.018900,-0.018900,0.102729,0.091200,78.60,298.492857,298.550000,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,0,2008,19,-0.018000,-0.012400,0.082043,0.072314,12.56,298.475714,298.557143,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,0,2008,20,-0.001500,-0.012400,0.151083,0.091529,3.66,299.455714,299.357143,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,0,2008,21,-0.001500,-0.019867,0.124329,0.125686,0.00,299.690000,299.728571,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,0,2008,22,0.056800,0.039833,0.062267,0.075914,0.76,299.780000,299.671429,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,1,2013,22,0.301471,0.380029,0.280629,0.383186,41.12,297.774286,298.964286,...,67.60,89.990000,41.12,17.185714,10.100000,27.400000,9.050000,32.6,21.8,33.0
412,1,2013,23,0.247600,0.296343,0.285371,0.350357,71.52,297.167143,298.328571,...,45.70,93.891429,71.52,17.448571,9.657143,27.520000,10.720000,33.8,21.4,68.0
413,1,2013,24,0.238729,0.251029,0.252586,0.249771,78.96,295.831429,296.607143,...,45.22,94.967143,78.96,16.410000,7.385714,27.200000,10.075000,32.6,21.6,93.2
414,1,2013,25,0.310429,0.302700,0.406614,0.403943,39.54,295.778571,297.400000,...,4.70,89.057143,39.54,15.137143,8.228571,26.700000,8.480000,32.2,21.8,34.1


In [571]:
test['train_or_test'] = 0

In [572]:
full_df = pd.concat([train, test])

In [573]:
full_df

,city,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,train_or_test
0,0,1990,18,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,1
1,0,1990,19,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,1
2,0,1990,20,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,1
3,0,1990,21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,1
4,0,1990,22,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,1,2013,22,0.301471,0.380029,0.280629,0.383186,41.12,297.774286,298.964286,...,89.990000,41.12,17.185714,10.100000,27.400000,9.050000,32.6,21.8,33.0,0
412,1,2013,23,0.247600,0.296343,0.285371,0.350357,71.52,297.167143,298.328571,...,93.891429,71.52,17.448571,9.657143,27.520000,10.720000,33.8,21.4,68.0,0
413,1,2013,24,0.238729,0.251029,0.252586,0.249771,78.96,295.831429,296.607143,...,94.967143,78.96,16.410000,7.385714,27.200000,10.075000,32.6,21.6,93.2,0
414,1,2013,25,0.310429,0.302700,0.406614,0.403943,39.54,295.778571,297.400000,...,89.057143,39.54,15.137143,8.228571,26.700000,8.480000,32.2,21.8,34.1,0


In [574]:
x_full_df = full_df
y_full_df = full_df.train_or_test

In [575]:
x_full_df = x_full_df.drop(['train_or_test'], axis=1);

In [576]:
from sklearn.linear_model import LinearRegression

In [577]:
reg = LinearRegression().fit(x_full_df, y_full_df)
reg.score(x_full_df, y_full_df)

0.5630372461200494

## Linear Regression

In [612]:
reg = LinearRegression().fit(x_train,y.total_cases)

In [619]:
reg.score(x_train,y.total_cases)

0.2095893690726912

In [620]:
reg.predict(x_train).round()

array([35., 41., 52., ..., -2., -5., -7.])

In [621]:
mean_absolute_error(y.total_cases, reg.predict(x_train).round())

21.022664835164836